# Multimodal Classification Testing

This notebook restores the multimodal network trained for grasp testset1 stored in `./dataset/grasp_testset1_logs` and test it.

## Dependencies

`Python 3.5.4` is used for development and following packages are required to run the code provided in the notebook:

`pip install googledrivedownloader`<br>
`pip install matplotlib`<br>
`pip install tensorflow-gpu`<br>
`pip install keras`<br>
`pip install numpy`

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#!pip install tensorflow==1.15.0
#!pip install keras==2.2.4

In [27]:
import pickle, os, csv, time, shutil
import json
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
# path="/content/drive/MyDrive/Deep-Multi-Sensory-Object-Categorization"
path = "/home/alex/Deep-Multi-Sensory-Object-Categorization"

In [4]:
def print_image(image, title):
    """Print the image

    :param image: image pixels in list
    :param title: title as string to be printed on top of the image
    """
    plt.imshow(image)
    plt.title(title)
    plt.colorbar()
    plt.show()

def time_taken(start, end):
    """Human readable time between `start` and `end`

    :param start: time.time()
    :param end: time.time()
    :returns: day:hour:minute:second
    """
    time = end-start
    day = time // (24 * 3600)
    time = time % (24 * 3600)
    hour = time // 3600
    time %= 3600
    minutes = time // 60
    time %= 60
    seconds = time
    day_hour_min_sec = str('%02d' % int(day))+":"+str('%02d' % int(hour))+":"+str('%02d' % int(minutes))+":"+str('%02d' % int(seconds))
    
    return day_hour_min_sec

## Video Dataset

In [5]:
file_0 = path+"/dataset/EMILver1_preprocessed/EMILver1_vi_vgg16fc2_pca19/arr_0.npy"
file_1 = path+"/dataset/EMILver1_preprocessed/EMILver1_vi_vgg16fc2_pca19/arr_1.npy"
file_2 = path+"/dataset/EMILver1_preprocessed/EMILver1_vi_vgg16fc2_pca19/arr_2.npy"
video_frames = np.load(file_0, allow_pickle=True)
action_label = np.load(file_1, allow_pickle=True)
object_label = np.load(file_2, allow_pickle=True)
for i in range(len(video_frames)):
    a01 = video_frames[i]
    while len(a01) < 658:
        a01 = np.concatenate((a01, np.zeros((1, a01.shape[1]))))
    video_frames[i] = a01
video_frames = np.array(list(video_frames))

In [6]:
video_frames[0].shape

(658, 19)

## Sound Dataset

In [7]:
file_0 = path+"/dataset/EMILver1_preprocessed/EMILver1_au_features/arr_0.npy"
file_1 = path+"/dataset/EMILver1_preprocessed/EMILver1_au_features/arr_1.npy"
file_2 = path+"/dataset/EMILver1_preprocessed/EMILver1_au_features/arr_2.npy"
audio_frames = np.load(file_0, allow_pickle=True)
action_label = np.load(file_1, allow_pickle=True)
object_label = np.load(file_2, allow_pickle=True)
for i in range(len(audio_frames)):
    a01 = audio_frames[i]
    while len(a01) < 658:
        a01 = np.concatenate((a01, np.zeros((1, a01.shape[1]))))
    audio_frames[i] = a01
audio_frames = np.array(list(audio_frames))

## Haptic Dataset

In [8]:
file_0 = path+"/dataset/EMILver1_preprocessed/EMILver1_sm_features/arr_0.npy"
file_1 = path+"/dataset/EMILver1_preprocessed/EMILver1_sm_features/arr_1.npy"
file_2 = path+"/dataset/EMILver1_preprocessed/EMILver1_sm_features/arr_2.npy"
haptic_frames = np.load(file_0, allow_pickle=True)
action_label = np.load(file_1, allow_pickle=True)
object_label = np.load(file_2, allow_pickle=True)
for i in range(len(haptic_frames)):
    a01 = haptic_frames[i]
    while len(a01) < 658:
        a01 = np.concatenate((a01, np.zeros((1, a01.shape[1]))))
    haptic_frames[i] = a01
haptic_frames = np.array(list(haptic_frames))

In [9]:
# one-hot encoding
num_classes = np.nanmax(action_label)+1
action_label_one_hot = np.zeros((len(action_label), num_classes)).astype(int)
for i in range(len(action_label)):
    action_label_one_hot[i, action_label[i]] = 1

num_classes = np.nanmax(object_label)+1
object_label_one_hot = np.zeros((len(object_label), num_classes)).astype(int)
for i in range(len(object_label)):
    object_label_one_hot[i, object_label[i]] = 1

# train-test-split
num_data = len(object_label)
train_id, test_id = train_test_split(np.array(range(num_data)), random_state=0)

## Building the Multimodal Network Architecture

<img src="pics/Multimodal.png" alt="drawing" width="400px"/>

## Multimodal Network Hyper-parameters

This network was trained for 300 epochs using Adam optimization with learning rate 1 x $10^{-4}$.

In [10]:
# Network hyper-parameters
batch = 1
training_epochs = 300
display_step = 1

behavior = "grasp"
testset = "testset1"
folder_name = behavior+'_'+testset
model_path = path+"/dataset/"+folder_name+"_logs/model.ckpt"
logs_path = path+"/dataset/"+folder_name+"_logs/"

# num_classes = category_label_train_one_hot.shape[1]
num_classes = object_label_one_hot.shape[1]

Y = tf.placeholder('float', [None, num_classes], name='LabelData')
print("Y: ", Y)

video_frames_max = 658
video_size = video_frames.shape[2]
video_X = tf.placeholder('float', [None, video_frames_max, video_size], name='InputData')

audio_frames_max = 658
audio_size = audio_frames.shape[2]
audio_keep_prob = tf.placeholder_with_default(1.0, shape=(), name='audio_keep')

haptic_frames_max = 658
haptic_size = haptic_frames.shape[2]
haptic_keep_prob = tf.placeholder_with_default(1.0, shape=(), name='haptic_keep')


Y:  Tensor("LabelData:0", shape=(?, 30), dtype=float32)



In [11]:
"""
Functions used to define models
"""

haptic_skip_2nd_maxpool = ["grasp", "hold", "low"]

def model(video_data_placeholder):
    with tf.name_scope("Model"):
        # Video
        net = tf.layers.flatten(video_data_placeholder)
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net)
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net)
        video_logits = tf.layers.dense(inputs=net, units=num_classes, activation=tf.nn.relu)
        
        # Audio
        audio_data_placeholder = tf.placeholder('float', [None, audio_frames_max, audio_size], name='audio_InputData')
        net = tf.layers.flatten(audio_data_placeholder)
        # Dense Layer
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net, rate=audio_keep_prob)
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net, rate=audio_keep_prob)
        audio_logits = tf.layers.dense(inputs=net, units=num_classes, activation=tf.nn.relu)
        
        # Haptic
        haptic_data_placeholder = tf.placeholder('float', [None, haptic_frames_max, haptic_size], name='haptic_InputData')
        net = tf.layers.flatten(haptic_data_placeholder)
        # Dense Layer
        net = tf.layers.dense(inputs=net, units=256, activation=tf.nn.relu)
        net = tf.layers.dropout(inputs=net, rate=haptic_keep_prob)
        haptic_logits = tf.layers.dense(inputs=net, units=num_classes, activation=tf.nn.relu)
        
        # Concatenate 
        logits = tf.concat([video_logits, audio_logits, haptic_logits], axis=1)
        logits = tf.nn.relu(logits)
        logits = tf.layers.dense(inputs=logits, units=num_classes)
        
    return logits


def loss(prediction, label_placeholder):
    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=label_placeholder))
        # Create a summary to monitor cost tensor
        cost_scalar = tf.summary.scalar("loss", cost)
    return cost, cost_scalar

def training(prediction, label_placeholder):
    with tf.name_scope('Optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(cost)
    return train_op

def evaluate(prediction, Y):
    with tf.name_scope('Accuracy'):
        # Test model
        correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
        # Create a summary to monitor accuracy tensor
        accuracy_scalar = tf.summary.scalar("accuracy", accuracy)
    return accuracy, accuracy_scalar

In [12]:
"""
Creating the Neural Network
"""

model_dict = {}
prediction = model(video_X)
model_dict["Model"] = prediction

cost, cost_scalar = loss(prediction, Y)
model_dict["Loss"] = cost
model_dict["Loss_scalar"] = cost_scalar

train_op = training(prediction, Y)
model_dict["Optimizer"] = train_op

eval_op, accuracy_scalar = evaluate(prediction, Y)
model_dict["Accuracy"] = eval_op
model_dict["Accuracy_scalar"] = accuracy_scalar

print("model_dict: ", model_dict)

# Initializing the variables
init = tf.global_variables_initializer()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver(max_to_keep=1)

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.dropout instead.


model_dict:  {'Model': <tf.Tensor 'Model/dense_8/BiasAdd:0' shape=(?, 30) dtype=float32>, 'Loss': <tf.Tensor 'Loss/Mean:0' shape=() dtype=float32>, 'Loss_scalar': <tf.Tensor 'Loss/loss:0' shape=() dtype=string>, 'Optimizer': <tf.Operation 'Optimizer/Adam' type=NoOp>, 'Accuracy': <tf.Tensor 'Accuracy/Mean:0' shape=() dtype=float32>, 'Accuracy_scalar': <tf.Tensor 'Accuracy/accuracy:0' shape=() dtype=string>}




## Testing for the whole clip

In [13]:
# Evaluate on test set

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, tf.train.latest_checkpoint(logs_path))
    
    avg_accuracy_list = 0.0
    total_batch = int(len(test_id)/batch)
    
    for start, end in zip(range(0, len(test_id), batch), range(batch, len(test_id)+1, batch)):
        video_input_data, label_data = video_frames[start:end], object_label_one_hot[start:end]

        audio_input_data = audio_frames[start:end]
        audio_X = tf.get_default_graph().get_tensor_by_name("Model/audio_InputData:0")

        haptic_input_data = haptic_frames[start:end]
        haptic_X = tf.get_default_graph().get_tensor_by_name("Model/haptic_InputData:0")
    
        accuracy = sess.run(model_dict["Accuracy"], 
                            feed_dict={
                                video_X: video_input_data, 
                                audio_X: audio_input_data, 
                                haptic_X: haptic_input_data, 
                                Y: label_data, 
                                audio_keep_prob: 1.0, 
                                haptic_keep_prob: 1.0
                            }
                           )
        
        avg_accuracy_list += accuracy/total_batch
            
    print("Overall Accuracy: ", avg_accuracy_list)

2021-12-02 17:27:28.476082: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-02 17:27:28.499478: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394450000 Hz
2021-12-02 17:27:28.499918: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a8f4523b80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-12-02 17:27:28.499948: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-7
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #157: KMP_AFFINITY: 8 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info 

INFO:tensorflow:Restoring parameters from /home/alex/Deep-Multi-Sensory-Object-Categorization/dataset/grasp_testset1_logs/model.ckpt-299



OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20053 thread 1 bound to OS proc set 2
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20052 thread 2 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20074 thread 3 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20075 thread 4 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20076 thread 5 bound to OS proc set 3
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20079 thread 8 bound to OS proc set 0
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20078 thread 7 bound to OS proc set 7
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20077 thread 6 bound to OS proc set 5
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20081 thread 10 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20082 thread 11 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20083 thread 12 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 20006 tid 20080 thread 9 bound to OS proc set 2
OMP: Info #25

Overall Accuracy:  0.8166666666666675


## Testing for every frame in video and for every 5th frame in audio and haptic

In [ ]:
# Predict after every frame in video and for every 5th frame in audio and haptic

all_frames_acc = {}
for a_frame in range(1, video_frames_max+1):
    all_frames_acc[a_frame] = [0, 0, 0] # correct, wrong, percentage

y_pred = {}
y_score = {}
for a_frame in range(1, video_frames_max+1):
    y_pred[a_frame] = []
    y_score[a_frame] = []
    
def fill_example(example, fill_frames, a_frame):
    temp = []
    temp.append(example[a_frame-1])
    for _ in range(fill_frames):
        example = np.concatenate((example, temp))
    return example

def give_me_complete_clip(num_of_frame, example, frame_list):
    fill_frames = frame_list - num_of_frame
    if fill_frames != 0:
        example = fill_example(example, fill_frames, num_of_frame)
    return example

category_labels = {"soft-blue-ball": 1, "soft-red-ball": 2, "soft-orange-ball": 3, "hard-pink-ball": 4, "light-red-car": 5, "blue-car": 6, "yellow-car": 7, "heavy-green-car": 8, "light-red-sponge": 9, "light-blue-sponge": 10, "blue-tissues": 11, "pink-tissues": 12, "soft-banana": 13, "light-hard-banana": 14, "heavy-hard-banana": 15, "hard-small-banana": 16, "green-frog": 17, "purple-duck": 18, "orange-fish": 19, "yellow-seal": 20, "small-yellow-die": 21, "big-yellow-die": 22, "light-soft-apple": 23, "light-hard-apple": 24, "heavy-soft-apple": 25, "heavy-hard-apple": 26, "heavy-blue-toy": 27, "red-toy": 28, "yellow-toy": 29, "heavy-green-toy": 30}

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, tf.train.latest_checkpoint(logs_path))
    
    for i_example in range(len(test_id)):
        print("Example Number: ", i_example+1)
        video_input_data, label_data = video_frames[test_id][i_example], object_label_one_hot[test_id][i_example]
        
        audio_input_data = audio_frames[test_id][i_example]
        
        haptic_input_data = haptic_frames[test_id][i_example]
        
        truth = np.argmax(label_data, 0)+1
        truth_cate = list(category_labels.keys())[list(category_labels.values()).index(truth)]
        print("True Category: ", truth_cate)
        
        for a_frame in range(1, video_frames_max+1):
            try:
                video_example = give_me_complete_clip(a_frame, video_input_data[0:a_frame], video_frames_max)
                video_temp = []
                video_temp.append(video_example)

                frame5Hz = a_frame*5 # For every frame in video there are 5 frames in audio and haptic

                audio_example = give_me_complete_clip(frame5Hz, audio_input_data[0:frame5Hz], audio_frames_max)
                audio_temp = []
                audio_temp.append(audio_example)

                haptic_example = give_me_complete_clip(frame5Hz, haptic_input_data[0:frame5Hz], haptic_frames_max)
                haptic_temp = []
                haptic_temp.append(haptic_example)

                audio_X = tf.get_default_graph().get_tensor_by_name("Model/audio_InputData:0")
                haptic_X = tf.get_default_graph().get_tensor_by_name("Model/haptic_InputData:0")

                predic = sess.run(model_dict["Model"], feed_dict={video_X: video_temp, audio_X: audio_temp, haptic_X: haptic_temp, audio_keep_prob: 1.0, haptic_keep_prob: 1.0})
                argmax = sess.run(tf.argmax(predic, 1))+1
                cate = list(category_labels.keys())[list(category_labels.values()).index(argmax)]
                print("Video Frame No.: ", a_frame, "Audio/Haptic Frame No.: ", frame5Hz, "Prediction: ", cate)

                y_pred[a_frame].extend(argmax)
                y_score[a_frame].extend(predic)

                if truth_cate == cate:
                    all_frames_acc[a_frame][0] += 1
                else:
                    all_frames_acc[a_frame][1] += 1
                all_frames_acc[a_frame][2] = all_frames_acc[a_frame][0]/(all_frames_acc[a_frame][0]+all_frames_acc[a_frame][1])
            except:
                pass
            
        print("X"*50)

INFO:tensorflow:Restoring parameters from /home/alex/Deep-Multi-Sensory-Object-Categorization/dataset/grasp_testset1_logs/model.ckpt-299
Example Number:  1
True Category:  hard-small-banana
Video Frame No.:  1 Audio/Haptic Frame No.:  5 Prediction:  heavy-hard-banana
Video Frame No.:  2 Audio/Haptic Frame No.:  10 Prediction:  heavy-hard-banana
Video Frame No.:  3 Audio/Haptic Frame No.:  15 Prediction:  light-hard-banana
Video Frame No.:  4 Audio/Haptic Frame No.:  20 Prediction:  light-hard-banana
Video Frame No.:  5 Audio/Haptic Frame No.:  25 Prediction:  light-hard-banana
Video Frame No.:  6 Audio/Haptic Frame No.:  30 Prediction:  light-hard-banana
Video Frame No.:  7 Audio/Haptic Frame No.:  35 Prediction:  light-hard-banana
Video Frame No.:  8 Audio/Haptic Frame No.:  40 Prediction:  light-hard-banana
Video Frame No.:  9 Audio/Haptic Frame No.:  45 Prediction:  light-hard-banana
Video Frame No.:  10 Audio/Haptic Frame No.:  50 Prediction:  light-hard-banana
Video Frame No.:  11

Video Frame No.:  107 Audio/Haptic Frame No.:  535 Prediction:  soft-banana
Video Frame No.:  108 Audio/Haptic Frame No.:  540 Prediction:  soft-banana
Video Frame No.:  109 Audio/Haptic Frame No.:  545 Prediction:  soft-banana
Video Frame No.:  110 Audio/Haptic Frame No.:  550 Prediction:  soft-banana
Video Frame No.:  111 Audio/Haptic Frame No.:  555 Prediction:  soft-banana
Video Frame No.:  112 Audio/Haptic Frame No.:  560 Prediction:  soft-banana
Video Frame No.:  113 Audio/Haptic Frame No.:  565 Prediction:  soft-banana
Video Frame No.:  114 Audio/Haptic Frame No.:  570 Prediction:  soft-banana
Video Frame No.:  115 Audio/Haptic Frame No.:  575 Prediction:  soft-banana
Video Frame No.:  116 Audio/Haptic Frame No.:  580 Prediction:  soft-banana
Video Frame No.:  117 Audio/Haptic Frame No.:  585 Prediction:  soft-banana
Video Frame No.:  118 Audio/Haptic Frame No.:  590 Prediction:  soft-banana
Video Frame No.:  119 Audio/Haptic Frame No.:  595 Prediction:  soft-banana
Video Frame 

Video Frame No.:  83 Audio/Haptic Frame No.:  415 Prediction:  orange-fish
Video Frame No.:  84 Audio/Haptic Frame No.:  420 Prediction:  blue-tissues
Video Frame No.:  85 Audio/Haptic Frame No.:  425 Prediction:  orange-fish
Video Frame No.:  86 Audio/Haptic Frame No.:  430 Prediction:  blue-tissues
Video Frame No.:  87 Audio/Haptic Frame No.:  435 Prediction:  blue-tissues
Video Frame No.:  88 Audio/Haptic Frame No.:  440 Prediction:  yellow-toy
Video Frame No.:  89 Audio/Haptic Frame No.:  445 Prediction:  yellow-toy
Video Frame No.:  90 Audio/Haptic Frame No.:  450 Prediction:  yellow-toy
Video Frame No.:  91 Audio/Haptic Frame No.:  455 Prediction:  yellow-toy
Video Frame No.:  92 Audio/Haptic Frame No.:  460 Prediction:  orange-fish
Video Frame No.:  93 Audio/Haptic Frame No.:  465 Prediction:  orange-fish
Video Frame No.:  94 Audio/Haptic Frame No.:  470 Prediction:  yellow-toy
Video Frame No.:  95 Audio/Haptic Frame No.:  475 Prediction:  yellow-toy
Video Frame No.:  96 Audio/H

Video Frame No.:  58 Audio/Haptic Frame No.:  290 Prediction:  light-red-car
Video Frame No.:  59 Audio/Haptic Frame No.:  295 Prediction:  light-red-car
Video Frame No.:  60 Audio/Haptic Frame No.:  300 Prediction:  light-red-car
Video Frame No.:  61 Audio/Haptic Frame No.:  305 Prediction:  light-red-car
Video Frame No.:  62 Audio/Haptic Frame No.:  310 Prediction:  light-red-car
Video Frame No.:  63 Audio/Haptic Frame No.:  315 Prediction:  light-red-car
Video Frame No.:  64 Audio/Haptic Frame No.:  320 Prediction:  light-red-car
Video Frame No.:  65 Audio/Haptic Frame No.:  325 Prediction:  light-red-car
Video Frame No.:  66 Audio/Haptic Frame No.:  330 Prediction:  light-red-car
Video Frame No.:  67 Audio/Haptic Frame No.:  335 Prediction:  light-red-car
Video Frame No.:  68 Audio/Haptic Frame No.:  340 Prediction:  light-red-car
Video Frame No.:  69 Audio/Haptic Frame No.:  345 Prediction:  big-yellow-die
Video Frame No.:  70 Audio/Haptic Frame No.:  350 Prediction:  big-yellow-d

Video Frame No.:  34 Audio/Haptic Frame No.:  170 Prediction:  blue-car
Video Frame No.:  35 Audio/Haptic Frame No.:  175 Prediction:  blue-car
Video Frame No.:  36 Audio/Haptic Frame No.:  180 Prediction:  blue-car
Video Frame No.:  37 Audio/Haptic Frame No.:  185 Prediction:  soft-banana
Video Frame No.:  38 Audio/Haptic Frame No.:  190 Prediction:  soft-banana
Video Frame No.:  39 Audio/Haptic Frame No.:  195 Prediction:  soft-banana
Video Frame No.:  40 Audio/Haptic Frame No.:  200 Prediction:  blue-car
Video Frame No.:  41 Audio/Haptic Frame No.:  205 Prediction:  blue-car
Video Frame No.:  42 Audio/Haptic Frame No.:  210 Prediction:  blue-car
Video Frame No.:  43 Audio/Haptic Frame No.:  215 Prediction:  blue-car
Video Frame No.:  44 Audio/Haptic Frame No.:  220 Prediction:  blue-car
Video Frame No.:  45 Audio/Haptic Frame No.:  225 Prediction:  blue-car
Video Frame No.:  46 Audio/Haptic Frame No.:  230 Prediction:  blue-car
Video Frame No.:  47 Audio/Haptic Frame No.:  235 Predi

...

Video Frame No.:  113 Audio/Haptic Frame No.:  565 Prediction:  heavy-hard-banana
Video Frame No.:  114 Audio/Haptic Frame No.:  570 Prediction:  heavy-hard-banana
Video Frame No.:  115 Audio/Haptic Frame No.:  575 Prediction:  heavy-hard-banana
Video Frame No.:  116 Audio/Haptic Frame No.:  580 Prediction:  heavy-hard-banana
Video Frame No.:  117 Audio/Haptic Frame No.:  585 Prediction:  heavy-hard-banana
Video Frame No.:  118 Audio/Haptic Frame No.:  590 Prediction:  heavy-hard-banana
Video Frame No.:  119 Audio/Haptic Frame No.:  595 Prediction:  heavy-hard-banana
Video Frame No.:  120 Audio/Haptic Frame No.:  600 Prediction:  heavy-hard-banana
Video Frame No.:  121 Audio/Haptic Frame No.:  605 Prediction:  heavy-hard-banana
Video Frame No.:  122 Audio/Haptic Frame No.:  610 Prediction:  heavy-hard-banana
Video Frame No.:  123 Audio/Haptic Frame No.:  615 Prediction:  heavy-hard-banana
Video Frame No.:  124 Audio/Haptic Frame No.:  620 Prediction:  heavy-hard-banana
Video Frame No.:

Video Frame No.:  85 Audio/Haptic Frame No.:  425 Prediction:  heavy-soft-apple
Video Frame No.:  86 Audio/Haptic Frame No.:  430 Prediction:  heavy-soft-apple
Video Frame No.:  87 Audio/Haptic Frame No.:  435 Prediction:  heavy-soft-apple
Video Frame No.:  88 Audio/Haptic Frame No.:  440 Prediction:  heavy-soft-apple
Video Frame No.:  89 Audio/Haptic Frame No.:  445 Prediction:  heavy-soft-apple
Video Frame No.:  90 Audio/Haptic Frame No.:  450 Prediction:  blue-car
Video Frame No.:  91 Audio/Haptic Frame No.:  455 Prediction:  heavy-soft-apple
Video Frame No.:  92 Audio/Haptic Frame No.:  460 Prediction:  heavy-soft-apple
Video Frame No.:  93 Audio/Haptic Frame No.:  465 Prediction:  orange-fish
Video Frame No.:  94 Audio/Haptic Frame No.:  470 Prediction:  heavy-soft-apple
Video Frame No.:  95 Audio/Haptic Frame No.:  475 Prediction:  heavy-soft-apple
Video Frame No.:  96 Audio/Haptic Frame No.:  480 Prediction:  heavy-soft-apple
Video Frame No.:  97 Audio/Haptic Frame No.:  485 Pre

Video Frame No.:  58 Audio/Haptic Frame No.:  290 Prediction:  heavy-green-car
Video Frame No.:  59 Audio/Haptic Frame No.:  295 Prediction:  heavy-green-car
Video Frame No.:  60 Audio/Haptic Frame No.:  300 Prediction:  heavy-green-car
Video Frame No.:  61 Audio/Haptic Frame No.:  305 Prediction:  heavy-green-car
Video Frame No.:  62 Audio/Haptic Frame No.:  310 Prediction:  heavy-green-car
Video Frame No.:  63 Audio/Haptic Frame No.:  315 Prediction:  heavy-green-car
Video Frame No.:  64 Audio/Haptic Frame No.:  320 Prediction:  heavy-green-car
Video Frame No.:  65 Audio/Haptic Frame No.:  325 Prediction:  heavy-green-car
Video Frame No.:  66 Audio/Haptic Frame No.:  330 Prediction:  heavy-green-car
Video Frame No.:  67 Audio/Haptic Frame No.:  335 Prediction:  heavy-green-car
Video Frame No.:  68 Audio/Haptic Frame No.:  340 Prediction:  heavy-green-car
Video Frame No.:  69 Audio/Haptic Frame No.:  345 Prediction:  heavy-green-car
Video Frame No.:  70 Audio/Haptic Frame No.:  350 Pr

Video Frame No.:  29 Audio/Haptic Frame No.:  145 Prediction:  blue-car
Video Frame No.:  30 Audio/Haptic Frame No.:  150 Prediction:  light-hard-banana
Video Frame No.:  31 Audio/Haptic Frame No.:  155 Prediction:  light-hard-banana
Video Frame No.:  32 Audio/Haptic Frame No.:  160 Prediction:  heavy-hard-banana
Video Frame No.:  33 Audio/Haptic Frame No.:  165 Prediction:  heavy-hard-banana
Video Frame No.:  34 Audio/Haptic Frame No.:  170 Prediction:  heavy-hard-banana
Video Frame No.:  35 Audio/Haptic Frame No.:  175 Prediction:  heavy-hard-banana
Video Frame No.:  36 Audio/Haptic Frame No.:  180 Prediction:  heavy-hard-banana
Video Frame No.:  37 Audio/Haptic Frame No.:  185 Prediction:  soft-banana
Video Frame No.:  38 Audio/Haptic Frame No.:  190 Prediction:  heavy-hard-banana
Video Frame No.:  39 Audio/Haptic Frame No.:  195 Prediction:  heavy-hard-banana
Video Frame No.:  40 Audio/Haptic Frame No.:  200 Prediction:  heavy-hard-banana
Video Frame No.:  41 Audio/Haptic Frame No.

Video Frame No.:  131 Audio/Haptic Frame No.:  655 Prediction:  heavy-hard-banana
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Example Number:  9
True Category:  blue-car
Video Frame No.:  1 Audio/Haptic Frame No.:  5 Prediction:  heavy-hard-banana
Video Frame No.:  2 Audio/Haptic Frame No.:  10 Prediction:  heavy-hard-banana
Video Frame No.:  3 Audio/Haptic Frame No.:  15 Prediction:  light-hard-banana
Video Frame No.:  4 Audio/Haptic Frame No.:  20 Prediction:  light-hard-banana
Video Frame No.:  5 Audio/Haptic Frame No.:  25 Prediction:  soft-orange-ball
Video Frame No.:  6 Audio/Haptic Frame No.:  30 Prediction:  light-blue-sponge
Video Frame No.:  7 Audio/Haptic Frame No.:  35 Prediction:  pink-tissues
Video Frame No.:  8 Audio/Haptic Frame No.:  40 Prediction:  soft-orange-ball
Video Frame No.:  9 Audio/Haptic Frame No.:  45 Prediction:  soft-orange-ball
Video Frame No.:  10 Audio/Haptic Frame No.:  50 Prediction:  heavy-green-car
Video Frame No.:  11 Audio/Haptic Frame No.

In [32]:
category_labels

'{"soft-blue-ball": 1, "soft-red-ball": 2, "soft-orange-ball": 3, "hard-pink-ball": 4, "light-red-car": 5, "blue-car": 6, "yellow-car": 7, "heavy-green-car": 8, "light-red-sponge": 9, "light-blue-sponge": 10, "blue-tissues": 11, "pink-tissues": 12, "soft-banana": 13, "light-hard-banana": 14, "heavy-hard-banana": 15, "hard-small-banana": 16, "green-frog": 17, "purple-duck": 18, "orange-fish": 19, "yellow-seal": 20, "small-yellow-die": 21, "big-yellow-die": 22, "light-soft-apple": 23, "light-hard-apple": 24, "heavy-soft-apple": 25, "heavy-hard-apple": 26, "heavy-blue-toy": 27, "red-toy": 28, "yellow-toy": 29, "heavy-green-toy": 30}'

In [19]:
"""
Writing a CSV file that records accuracy for each frame
"""

with open(logs_path+folder_name+"_acc_VEveryFrame_AH5Frame.csv",'w') as f:
    writer = csv.writer(f, lineterminator="\n")
    
    first_row = ["Frame_No", "Correct", "Wrong", "Accuracy"]
    writer.writerow(first_row)
    for a_frame in range(1, video_frames_max+1):
        row = [a_frame]
        row.append(all_frames_acc[a_frame][0])
        row.append(all_frames_acc[a_frame][1])
        row.append(all_frames_acc[a_frame][2])
        writer.writerow(row)